# Deploy NCF model and create Endpoint in SageMaker

This notebook uses <b>`conda_python3`</b> as the default kernel.

## 0. Setting Environment

In [1]:
import boto3
import time
import os
import sagemaker
from datetime import datetime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session  = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker.session.Session().default_bucket()

print("role: ", role)
print("bucket: ", bucket)

role:  arn:aws:iam::250476343008:role/dynamic-ab-testing-stack-SageMakerRole-mrq20NtVhMDX
bucket:  sagemaker-ap-northeast-2-250476343008


### Upload dataset to S3

In [3]:
item_meta_bucket='item-meta-joe-0621' # replace with the name of your S3 bucket
data_filename = "dataset/merged_data.csv"

response_upload = boto3.Session().resource('s3').Bucket(item_meta_bucket).Object(data_filename).upload_file(data_filename)

s3_its_filename = "s3://{}/{}".format(item_meta_bucket, data_filename)

print("s3_merged_data_filename: \n", s3_its_filename)

s3_merged_data_filename: 
 s3://item-meta-joe-0621/dataset/merged_data.csv


## 1. Upload the model (model.tar.gz) to S3

In [4]:
model_prefix = 'ncf/model'
model_filename = "./model/model.tar.gz"

model_s3_path = sagemaker_session.upload_data(model_filename, bucket, model_prefix)
print("model: \n", model_s3_path)

model: 
 s3://sagemaker-ap-northeast-2-250476343008/ncf/model/model.tar.gz


## 2. Create a model in SageMaker
Register in Model Registry

In [5]:
# Define model name
model_name = 'ncf-tf-model'

In [6]:
# Get the current session's region
session = boto3.Session()
region = session.region_name

print("Current region:", region)

Current region: ap-northeast-2


<br>
The model to register (model.tar.gz) was written in the TensorFlow 2.6 environment.<br>
So the container image uses 'tensorflow-inference:2.6-cpu'.

In [7]:
sagemaker_client = boto3.client('sagemaker', region_name=region)

# Create a model in SageMaker
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': '763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference:2.6-cpu'.format(region),
        'ModelDataUrl': 's3://{}/{}'.format(bucket, model_prefix + '/model.tar.gz'),
    }
)

## 3. Create SageMaker Endpoint

In [8]:
# Define the endpoint config and endpoint names
endpoint_config_name = 'ncf-model-endpoint-config'
endpoint_name = 'ncf-model-endpoint'

In [9]:
# Create endpoint configuration
create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.xlarge',  # Choose the desired instance type
            'InitialVariantWeight': 1
        }
    ]
)

print("Endpoint config created:", create_endpoint_config_response['EndpointConfigArn'])

Endpoint config created: arn:aws:sagemaker:ap-northeast-2:250476343008:endpoint-config/ncf-model-endpoint-config


In [10]:
# Create endpoint
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print("Creating endpoint...")

Creating endpoint...


####Wait until Endpoint creation is complete

In [11]:
# Wait for the endpoint to be in service
endpoint_status = 'Creating'
while endpoint_status == 'Creating':
    time.sleep(30)
    endpoint_status = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)['EndpointStatus']
    print("Endpoint status:", endpoint_status)

print("Endpoint created:", create_endpoint_response['EndpointArn'])

Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService
Endpoint created: arn:aws:sagemaker:ap-northeast-2:250476343008:endpoint/ncf-model-endpoint


# You can make inference requests to the SageMaker Endpoint ARNs below.
NCF model inference in Lambda Function uses the SageMaker Endpoint ARN below.

In [12]:
print("SageMaker Endpoint ARN : ", create_endpoint_response['EndpointArn'])

SageMaker Endpoint ARN :  arn:aws:sagemaker:ap-northeast-2:250476343008:endpoint/ncf-model-endpoint


#### save variable
Save variables needed for clean-up

In [13]:
%store endpoint_config_name
%store endpoint_name
%store model_name

Stored 'endpoint_config_name' (str)
Stored 'endpoint_name' (str)
Stored 'model_name' (str)


## (Optional) Inference Test
In the code below, put values ​​such as 1, 2, 100, etc. into user_id to check if 10 recommended items (item_id) are displayed well.

### a. Output ITEM_ID (10 items) for the input user_id

In [14]:
import numpy as np
import pandas as pd
import json
import sagemaker

# Load dataset and create user_to_index and item_to_index dictionaries
data = pd.read_csv('./dataset/merged_data.csv') 
user_ids = data['user_id'].unique()
item_ids = data.groupby('item_id').size().sort_values(ascending=False).index.to_numpy()

user_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
item_to_index = {item_id: index for index, item_id in enumerate(item_ids)}

# Enter USER_ID to infer
user_id = 3

user_idx = user_to_index[user_id]
item_idx_list = np.array([item_to_index[item_id] for item_id in item_ids])
user_input = np.full(len(item_ids), user_idx).reshape(-1, 1)
item_input = item_idx_list.reshape(-1, 1)


# Define SageMaker client
sagemaker_client = boto3.client('sagemaker-runtime')

# Perform inference using invoke_endpoint()
response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(
        {"user_input": user_input.tolist(),
         "item_input": item_input.tolist()}
    )
)

# Parse response
predictions = json.loads(response['Body'].read().decode('utf-8'))

# Convert predictions to a 1-dimensional array
predictions_array = np.array(predictions['predictions']).reshape(-1)

# Get the indices of the top 10 items
top_10_indices = np.argsort(predictions_array)[-10:][::-1]

# Get the item_ids for the top 10 items
top_10_item_ids = [item_ids[idx] for idx in top_10_indices]

print("Top 10 item IDs for user ID", user_id, ":\n", top_10_item_ids)

Top 10 item IDs for user ID 3 :
 ['1849aebd-fdda-4f05-be3f-28f2c61d8901', '471622d6-3300-4591-816b-ab8487e70274', '78080d05-b078-441f-b245-54b2a2dec872', '3fa257ab-5c4c-4963-941f-bc73bafc8bff', '8d67146f-0b59-4761-9120-34ebfee7b299', '07441beb-9b1a-4c25-b302-2f11362cff86', '1f7340f0-805d-4ba9-a0e4-76d392eab5a1', '0770ffd6-c89a-489a-b9dc-e5c1c267e102', '95be8b94-feeb-48df-8449-34924e4d849d', 'ca3f03ff-72a5-4a6a-9cd9-a513fcde604e']


### b. Output ITEM_ID and detailed information (10 items) for the input user_id together

In [15]:
def get_item_list_details(items_df, item_id_list):
    '''
    Provides additional information of the corresponding ITEM_ID
    '''
    df = pd.DataFrame(data={'ITEM_ID':item_id_list})
    rec_item_df = df.merge(items_df)
    return rec_item_df

In [16]:
import numpy as np
import pandas as pd
import json
import sagemaker

# Load dataset and create user_to_index and item_to_index dictionaries
data = pd.read_csv('./dataset/merged_data.csv') 
user_ids = data['user_id'].unique()
item_ids = data.groupby('item_id').size().sort_values(ascending=False).index.to_numpy()

user_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
item_to_index = {item_id: index for index, item_id in enumerate(item_ids)}

# Enter USER_ID to infer
user_id = 3

user_idx = user_to_index[user_id]
item_idx_list = np.array([item_to_index[item_id] for item_id in item_ids])
user_input = np.full(len(item_ids), user_idx).reshape(-1, 1)
item_input = item_idx_list.reshape(-1, 1)


# Define SageMaker client
sagemaker_client = boto3.client('sagemaker-runtime')

# Perform inference using invoke_endpoint()
response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(
        {"user_input": user_input.tolist(),
         "item_input": item_input.tolist()}
    )
)

# Parse response
predictions = json.loads(response['Body'].read().decode('utf-8'))

# Convert predictions to a 1-dimensional array
predictions_array = np.array(predictions['predictions']).reshape(-1)

# Get the indices of the top 10 items
top_10_indices = np.argsort(predictions_array)[-10:][::-1]

# Get the item_ids for the top 10 items
top_10_item_ids = [item_ids[idx] for idx in top_10_indices]

print("Top 10 item IDs for user ID", user_id, ":\n")

## join with 'training_item.csv'
items_df = pd.read_csv("./dataset/training_item.csv")
get_item_list_details(items_df, top_10_item_ids)

Top 10 item IDs for user ID 3 :



,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,1849aebd-fdda-4f05-be3f-28f2c61d8901,Black Rain Jacket,apparel,jacket,Well-insulated rain jacket for men,122.99
1,471622d6-3300-4591-816b-ab8487e70274,Coffee Maker,housewares,kitchen,A must-have for your kitchen,55.99
2,78080d05-b078-441f-b245-54b2a2dec872,Sophisticated Ceramic Bowl,housewares,bowls,Sophisticated ceramic bowl for every occasion,18.99
3,3fa257ab-5c4c-4963-941f-bc73bafc8bff,Squash,groceries,vegetables,Keep your pantry well stocked with this delish...,5.99
4,8d67146f-0b59-4761-9120-34ebfee7b299,Beef,groceries,meat,Our beef is raised sustainably by our farmers,11.99
5,07441beb-9b1a-4c25-b302-2f11362cff86,Supercool Belt,accessories,belt,Supercool belt for women,42.99
6,1f7340f0-805d-4ba9-a0e4-76d392eab5a1,Chef Knife,housewares,kitchen,Incomparable for every kitchen,65.99
7,0770ffd6-c89a-489a-b9dc-e5c1c267e102,Beef,groceries,meat,Our beef is organic and raised sustainably,14.99
8,95be8b94-feeb-48df-8449-34924e4d849d,Faultless Microphone,instruments,microphone,This faultless microphone can be relied upon i...,129.99
9,ca3f03ff-72a5-4a6a-9cd9-a513fcde604e,Supercool Glasses,accessories,glasses,These supercool glasses for men are incomparable,118.99
